In [6]:
import matplotlib.pyplot as plt
import PIL
import PIL.Image as Image
import PIL.ImageDraw2 as ImageDraw
import io
import tensorflow as tf

In [4]:
file='/home/jupyter/data/tfrecord/poletag/25-images-27-aug-yellow-validation.record'
rawdata=tf.data.TFRecordDataset(file)

In [9]:
for record,rawrecord in enumerate(rawdata.as_numpy_iterator()):
    example=tf.train.Example()
    example.ParseFromString(rawrecord)
    rawdict=dict(example.features.feature)
    
    imgbytes=rawdict['image/encoded'].bytes_list.value[0]
    img=Image.open(io.BytesIO(imgbytes))
    
    width=rawdict['image/width'].int64_list.value[0]
    height=rawdict['image/height'].int64_list.value[0]
    print(f"Overlaying Annotations for Image:{rawdict['image/filename'].bytes_list.value[0].decode()}")
    for idx,tag in enumerate(rawdict['image/object/class/text'].bytes_list.value):
        minxy=(int(rawdict['image/object/bbox/xmin'].float_list.value[idx]*width),
               int(rawdict['image/object/bbox/ymin'].float_list.value[idx]*height))
        maxxy=(int(rawdict['image/object/bbox/xmax'].float_list.value[idx]*width),
               int(rawdict['image/object/bbox/ymax'].float_list.value[idx]*height))
        img_overlay=PIL.ImageDraw.Draw(img)
        img_overlay.rectangle([minxy,maxxy],width=3)
    img.save(rawdict['image/filename'].bytes_list.value[0].decode())

Overlaying Annotations for Image:1008141E-Stephen.Olivera@sce.com-23D55386-D142-466B-A0CD-B429431B35E4.jpg
Overlaying Annotations for Image:1009059E-Arturo.Plascencia@sce.com-450F9C1E-20CD-49B3-8905-5EB01C4AB7B2.jpg
Overlaying Annotations for Image:1011606E-Airion.Piggue@sce.com-A891CFDA-4F61-462C-9055-98CA6DE46209.jpg
Overlaying Annotations for Image:1008120E-John.Anderson@sce.com-44B73339-E421-47DB-9AAE-A665BD714F89.jpg
Overlaying Annotations for Image:1007364E-Samuel.Cardoza@sce.com-61183F5E-88F9-4953-8318-35F9CE30946C.jpg
Overlaying Annotations for Image:1008916E-Arturo.Plascencia@sce.com-65461FCF-1F3A-40AF-92C1-4F86E703835E.jpg
Overlaying Annotations for Image:1009062E-Arturo.Plascencia@sce.com-0EAE92AC-3C9A-4D2E-95B5-E187C585B1CC.jpg
Overlaying Annotations for Image:1008542E-Caesar.Sandoval@sce.com-3D2B1CC9-D925-4037-B368-80DC55D3A176.jpg
Overlaying Annotations for Image:1008115E-Stephen.Olivera@sce.com-83294FC2-E925-4D25-8F68-3C21815471B8.jpg
Overlaying Annotations for Image:100